In [10]:
class CNF:
    def __init__(self, n_var, n_clauses, clauses):
        self.var_map = {}
        self.n_var = n_var
        self.n_clauses = n_clauses
        self.clauses = clauses
        self.satisfiable = 0 # unknown -> 0, sat -> 1, unsat -> 2

In [11]:
def make_formula(filename):
    f = open(filename)
    lines = f.readlines()
    flag = 0
    clauses = []
    n_var = 0
    n_clauses = 0
    for line in lines:
        if (flag == 0):
            if (line[0] == 'p'):
                flag = 1
                info = line.split()
                n_var = info[2]
                n_clauses = info[3]
        else:
            clause = list(map(int, line.split()[:-1]))
            clauses.append(clause)
    return CNF(n_var, n_clauses, clauses)


In [12]:
cnf = make_formula('test1.cnf')
cnf2 = do_pure_literal(cnf)
print(cnf2.clauses)

[[1, -5]]
DD


In [13]:
def assign_value(n):
    if (n < 0):
        return 0
    else:
        return 1

In [14]:
def find_var_in_clause(clause, var):
    for v in clause:
        if abs(v) == var:
            if (v < 0):
                return 0
            else:
                return 1

In [15]:
def resolution_for_var(c1, c2, var):
    new_c1 = c1[:]
    new_c2 = c2[:]
    if var in c1:
        new_c1.remove(var)
    else:
        new_c1.remove(-var)
    if var in c2:
        new_c2.remove(var)
    else:
        new_c2.remove(-var)
    return list(set(new_c1 + new_c2))

In [22]:
def assign_val_to_var(var_map, clauses):
    new_clauses_edit = clauses[:]
    for var in var_map.keys():
        for clause in clauses:
            clause_search = list(map(abs, clause[:]))
            if var in clause_search:
                assign = cnf.var_map[var]
                if (assign == find_var_in_clause(clause, var)):
                    new_clauses_edit.remove(clause)
                else:
                    index = new_clauses_edit.index(clause)
                    if assign == 1:
                        v = -var
                    else:
                        v = var
                    new_clauses_edit[index].remove(v)

In [17]:
def do_pure_literal(cnf):
    clauses = cnf.clauses
    new_clauses = []
    new_clauses_edit = []
    unit_var = 0
    unit_index = []
    for i, clause in enumerate(clauses):
        if (len(clause) == 1):
            unit_var = clause[0]
            if (unit_var < 0):
                if not (check_var_map(cnf.var_map, -unit_var, 0)):
                    cnf.satisfiable = 2
                cnf.var_map[-unit_var] = 0
                unit_index.append(i)
            else:
                if not (check_var_map(cnf.var_map, unit_var, 1)):
                    cnf.satisfiable = 2
                cnf.var_map[unit_var] = 1
                unit_index.append(i)
    for i in range(len(clauses)):
        if not (i in unit_index):
            new_clauses.append(clauses[i])
    new_clauses_edit = new_clauses[:]
    for var in cnf.var_map.keys():
        for clause in new_clauses:
            clause_search = list(map(abs, clause[:]))
            if var in clause_search:
                assign = cnf.var_map[var]
                if (assign == find_var_in_clause(clause, var)):
                    new_clauses_edit.remove(clause)
                else:
                    index = new_clauses_edit.index(clause)
                    if assign == 1:
                        v = -var
                    else:
                        v = var
                    new_clauses_edit[index].remove(v)
    cnf.clauses = new_clauses_edit
    cnf.n_clauses = len(new_clauses)
    if (cnf.n_clauses == 0):
        cnf.satisfiable = 1
    return cnf

In [18]:
def check_var_map(a, var, val):
    if (var in a):
        if (a[var] != val):
            return False
        else:
            return True
    else:
        return True

In [32]:
def find_conflict(clauses, assign_map):
    for clause in clauses:
        result = assign_clause(clause, assign_map)
        if result == 0:
            return clause
    return []

In [33]:
def assign_clause(clause, assign_map):
    # 0 -> false, 1 -> True, 2 -> Unknown
    result = 0
    for var in clause:
        exist, assign = find_assign_value(var, assign_map)
        if not exist:
            return 2
        if assign == True:
            if var > 0:
                return 1
        else:
            if var < 0:
                return 1
    return result

In [31]:
def dpll(clauses, assign_map):
    result, new_assign_map = assign_to_unit_clause(clauses, assign_map)
    if result == 1:
        return True, new_assign_map
    clause = find_conflict(result, new_assign_map)
    if result == 2:
        learned_clause = learning_procedure(clause, clauses)
        new_clauses = clauses + learned_clause
        if learned_clause == []:
            return False, assign_map
        else:
            old_assign = backtrack(learned_clause, assign_map)
            dpll(new_clauses, old_assign)
    else:
        assign_map = make_decision(clauses, assign_map)
        dpll(clauses, assign_map)
        

In [27]:
def assign_result(clauses, assign_map):
    new_clauses = []
    for clause in clauses:
        res = 0
        clause_result = []
        for i, var in enumerate(clause):
            t = (abs(var), abs(var) == var)
            exist, val = find_assign_value(t[0], assign_map)
            if exist:
                if val == t[1]:
                    res = 1
                    break
            else:
                clause_result.append(var)
        if res == 0:
            new_clauses.append(clause_result)
    return new_clauses

In [26]:
def find_assign_value(var, assign_map):
    for v in assign_map:
        if v[0] == var:
            return True, v[1]
    return False, 0

In [28]:
def assign_to_unit_clause(clauses, assign_map):
    result = assign_result(clauses, assign_map)
    if result == []:
        return 1, assign_map
    while exist_unit_clause(result):
        for clause in result:
            if len(clause) == 1:
                var = clause[0]
                new_assign_map = add_assign_map(abs(var), assign_value(var))
                ind, assign = assign_to_unit_clause(clauses, new_assign_map)
    if exist_false(result):
        return 2, assign_map
    else:
        return 1, assign_map


In [29]:
def exist_false(clauses):
    for clause in clauses:
        if clause == []:
            return True
    return False

In [30]:
def algorithm(cnf):
    b, assign = dpll(cnf.clauses, [])
    if b:
        print("s SATISFIABLE")
    else:
        print("s UNSATISFIABLE")

In [19]:
f = open('test1.cnf')
lines = f.readlines()
flag = 0
clauses = []
for i, line in enumerate(lines):
    if (flag == 0):
        print(line[0])
        if (line[0] == 'p'):
            flag = 1
            k = line.split()
            print(k)
    else:
        clause = list(map(int, line.split()))
        clause = clause[:-1]
        clauses.append(clause)
print(clauses)
        

c
c
p
['p', 'cnf', '5', '3']
[[1, -5, 4], [-1, 5, 3, -4], [-4]]


In [25]:
a = None
b = []
b.append(a)
print(b)

[None]
